In [1]:
import nimblend as nd
import numpy as np

In [2]:
# Create two arrays with different coordinates
data1 = np.array([[1, 2], [3, 4]])
coords1 = {'x': ['a', 'b'], 'y': [0, 1]}
arr1 = nd.Array(data1, coords1)

data2 = np.array([[10, 20, 30], [40, 50, 60]])
coords2 = {'x': ['b', 'c'], 'y': [0, 1, 2]}
arr2 = nd.Array(data2, coords2)

# Operation automatically aligns coordinates
result = arr1 + arr2
print(result)

# Create a lazy array with Dask
lazy_arr = nd.Array(data1, coords1, chunks="auto")
print(lazy_arr.is_lazy)  # True

# Save array to Zarr format
nd.to_zarr(result, "result.zarr")

# Load from Zarr
loaded = nd.from_zarr("result.zarr")

Array((3, 3), {'y': (3,), 'x': (3,)})
True


/home/vscode/.local/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/vscode/.local/lib/python3.12/site-packages/zarr/core/array.py:3989: UserWarning: The dtype `<U1` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/home/vscode/.local/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
